In [1]:
import pandas as pd
import random
from sklearn.neighbors import KNeighborsClassifier

In [2]:
matriz_entrenamiento= pd.read_csv('matriz.csv')

In [14]:
matriz_entrenamiento.head()

,postulacion,nombre_zona,tipo_de_trabajo,nivel_laboral,nombre_area,sexo,nombre,estado,edad
0,0,7,1,2,153,2,5,2,34
1,0,7,1,4,160,2,5,2,34
2,1,7,1,4,143,2,5,2,34
3,1,7,1,2,143,2,5,2,34
4,1,7,1,4,5,2,5,2,34


In [11]:
 matriz_entrenamiento.drop(columns='Unnamed: 0',axis=1,inplace=True)

In [40]:
matrix= matriz_entrenamiento

In [24]:
print(matrix)

          postulacion  nombre_zona  tipo_de_trabajo  nivel_laboral  \
0                   0            7                1              2   
1                   0            7                1              4   
2                   1            7                1              4   
3                   1            7                1              2   
4                   1            7                1              4   
5                   0            7                1              4   
6                   1            7                1              2   
7                   0            7                1              4   
8                   1            7                5              4   
9                   1            7                1              4   
10                  0            1                1              1   
11                  0            7                1              2   
12                  1            7                1              4   
13                  

In [41]:
matrix= matrix[ matrix['postulacion']==1 ]


In [12]:
matrix=matrix.reset_index()

In [43]:
matrix.drop(columns='index',axis=1,inplace=True)

In [44]:
matrix.head()

,postulacion,nombre_zona,tipo_de_trabajo,nivel_laboral,nombre_area,sexo,nombre,estado,edad
0,1,7,1,4,143,2,5,2,34
1,1,7,1,2,143,2,5,2,34
2,1,7,1,4,5,2,5,2,34
3,1,7,1,2,143,2,5,2,34
4,1,7,5,4,119,2,5,2,34


# KNN

In [45]:
knn = KNeighborsClassifier(n_neighbors= 5)

In [47]:
y = matrix['postulacion'].head(500000)
X = matrix.head(500000).drop('postulacion',axis=1)

In [50]:
y.head()

0    1
1    1
2    1
3    1
4    1
Name: postulacion, dtype: int64

In [51]:
X.head()

,nombre_zona,tipo_de_trabajo,nivel_laboral,nombre_area,sexo,nombre,estado,edad
0,7,1,4,143,2,5,2,34
1,7,1,2,143,2,5,2,34
2,7,1,4,5,2,5,2,34
3,7,1,2,143,2,5,2,34
4,7,5,4,119,2,5,2,34


In [52]:
knn.fit(X, y)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=5, p=2,
           weights='uniform')

In [42]:
test_final_df = pd.read_csv('test_final_100k.csv')

In [43]:
postulantes_filtrados_df= pd.read_csv('postulantes_filtrados_df')


In [44]:
test_final_df.drop_duplicates('idpostulante').merge(postulantes_filtrados_df,on='idpostulante',how='inner').shape

(64646, 8)

In [45]:
avisos_filtrados= pd.read_csv('avisos_detalle_filtrados')

In [46]:
test_final_df = test_final_df.merge(avisos_filtrados,on='idaviso',how='left')

In [47]:
postulantes_filtrados_df.drop(columns='Unnamed: 0',axis=1,inplace=True)

In [48]:
postulantes_filtrados_df.head()

,idpostulante,sexo,nombre,estado,edad
0,6MM,2,2,3,33
1,Nzz,3,2,3,100
2,ZX1,3,2,3,100
3,Nq5,3,2,3,100
4,ebE,2,7,2,66


In [49]:
test_final_df = test_final_df.merge(postulantes_filtrados_df,on='idpostulante',how='inner')

In [50]:
test_final_df = test_final_df.drop(['id','Unnamed: 0','idaviso','idpostulante'],axis=1)

In [51]:
test_final_df.head()

,nombre_zona,tipo_de_trabajo,nivel_laboral,nombre_area,sexo,nombre,estado,edad
0,1,1,1,30,1,0,2,42
1,1,1,1,30,2,3,1,31
2,1,1,1,30,1,7,2,36
3,1,1,3,170,2,4,2,69
4,1,1,3,170,1,6,1,32


In [107]:
postulantes_filtrados_df.columns

Index(['idpostulante', 'sexo', 'nombre', 'estado', 'edad'], dtype='object')

In [111]:
test_final_df.shape

(100000, 8)

In [112]:
pred = knn.predict(test_final_df)

In [68]:
X.shape

(500000, 8)

In [69]:
test_final_df.shape

(100000, 10)

In [118]:
df = pd.DataFrame({'A': pred })

In [125]:
df['A'].value_counts()

1    100000
Name: A, dtype: int64

# Pruebo bagging

In [32]:
from sklearn import model_selection
from sklearn.model_selection import train_test_split
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier

In [15]:
matrix=matrix.reset_index()

In [17]:
matrix.drop(columns='index',axis=1,inplace=True)

In [18]:
matrix = matriz_entrenamiento[matriz_entrenamiento['postulacion'] == 0].head(300000).append(matriz_entrenamiento[matriz_entrenamiento['postulacion'] == 1].head(250000))
array= matrix.values

In [54]:
matrix.columns

Index(['postulacion', 'nombre_zona', 'tipo_de_trabajo', 'nivel_laboral',
       'nombre_area', 'sexo', 'nombre', 'estado', 'edad'],
      dtype='object')

In [ ]:
X = array[:,1:8]
Y = array[:,0]

In [25]:
seed = 7
kfold = model_selection.KFold(n_splits=10, random_state=seed)

In [26]:
cart = DecisionTreeClassifier()

In [27]:
num_trees = 35
model = BaggingClassifier(base_estimator=cart, n_estimators=num_trees, random_state=seed)

In [28]:
results = model_selection.cross_val_score(model, X, Y, cv=kfold)
print(results.mean())

0.57077


# Otra forma de bagging

In [60]:
df_X = array[:,1:9]
df_Y = array[:,0]

In [80]:
x_train, x_test, y_train , y_test= train_test_split(df_X,df_Y, test_size=0.3, random_state=4)

In [81]:
bg= BaggingClassifier( base_estimator=cart, max_samples= 0.5, max_features=1.0, n_estimators=20) 

In [82]:
bg.fit(x_train, y_train)

BaggingClassifier(base_estimator=DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best'),
         bootstrap=True, bootstrap_features=False, max_features=1.0,
         max_samples=0.5, n_estimators=20, n_jobs=1, oob_score=False,
         random_state=None, verbose=0, warm_start=False)

In [83]:
bg.score(x_test,y_test) 

0.7765666666666666

In [84]:
pred= bg.predict( test_final_df)

In [85]:
pred

array([0, 0, 0, ..., 0, 0, 0], dtype=int64)

In [61]:
df_X[1]

array([  7,   1,   4, 160,   2,   5,   2,  34], dtype=int64)

In [62]:
matrix.head()

,postulacion,nombre_zona,tipo_de_trabajo,nivel_laboral,nombre_area,sexo,nombre,estado,edad
0,0,7,1,2,153,2,5,2,34
1,0,7,1,4,160,2,5,2,34
5,0,7,1,4,63,2,5,2,34
7,0,7,1,4,96,2,5,2,34
10,0,1,1,1,137,2,5,2,34


In [86]:
test_final = pd.read_csv('test_final_100k.csv')

In [91]:
test_final['sepostulo']= pd.DataFrame(pred)

In [92]:
submit = test_final.drop(['idaviso','idpostulante'],axis=1)

In [93]:
submit.head()

,id,sepostulo
0,0,0
1,1,0
2,2,0
3,3,0
4,4,1


In [94]:
submit.sepostulo.value_counts()

0    65604
1    34396
Name: sepostulo, dtype: int64

In [95]:
submit.to_csv('Submit_file')